<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Thu Jun 25 16:12:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [5]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-06-25T16-12-10'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 111 (delta 1), reused 65 (delta 1), pack-reused 0
Receiving objects: 100% (111/111), 1.20 MiB | 1.34 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [7]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-06-25T16-12-10
total 69
drwx------  4 root root  4096 Jun 25 16:12 data
-rw-------  1 root root 35149 Jun 25 16:12 LICENSE
drwx------  2 root root  4096 Jun 25 16:12 notebooks
-rw-------  1 root root  9745 Jun 25 16:12 README.md
drwx------  3 root root  4096 Jun 25 16:12 reports
drwx------  2 root root  4096 Jun 25 16:12 requirements
drwx------ 13 root root  4096 Jun 25 16:12 src
drwx------  3 root root  4096 Jun 25 16:12 vscode-extension


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-06-25T16-12-10/.env'

In [12]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'dir_data': '../data/processed/subtoken/',
  'file_checkpoint_dir': 'models/checkpoints/baseline/',
  'dir_preprocessed_data': '../data/processed/seq2seq/',
  'max_input_length': 128,
  'max_output_length': 8,
  'input_vocab_size': 5000,
  'input_embedding_dim': 50,
  'output_vocab_size': 5000,
  'output_embedding_dim': 50,
  'latent_dim': 320,
  'learning_rate': 0.0001,
  'epochs': 50,
  'batch_size': 512,
  'random_seed': 1,
  'eval_averaging': 'macro'
}

run(Namespace(**params))

Initializing logger


wandb: Wandb version 0.9.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Experiment parameters:  Namespace(batch_size=512, dir_data='../data/processed/subtoken/', dir_preprocessed_data='../data/processed/seq2seq/', epochs=50, eval_averaging='macro', file_checkpoint_dir='models/checkpoints/baseline/', input_embedding_dim=50, input_vocab_size=5000, latent_dim=320, learning_rate=0.0001, max_input_length=128, max_output_length=8, output_embedding_dim=50, output_vocab_size=5000, random_seed=1)
Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderEmbedding (Embedding) multiple                  250000    
_________________________________________________________________
EncoderLSTM (LSTM)           multiple                  474880    
Total params: 724,880
Trainable params: 724,880
Non-trainable params: 0
____________________________________________________

wandb: Visualizing heatmap.


epoch 1 evaluation time: 133.86270380020142 sec
epoch evaluation:  {'epoch': 1, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=2.6290793>, 'epoch_test_sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.18866861>, 'epoch_test_precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.0025760953>, 'epoch_test_recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.0030731345>, 'epoch_test_f1': <tf.Tensor: shape=(), dtype=float32, numpy=0.0021288928>}


...

wandb: Visualizing heatmap.


epoch 6 evaluation time: 142.88788414001465 sec
epoch evaluation:  {'epoch': 6, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=1.6565787>, 'epoch_test_sparse_categorical_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.27440318>, 'epoch_test_precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.30293065>, 'epoch_test_recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.15967456>, 'epoch_test_f1': <tf.Tensor: shape=(), dtype=float32, numpy=0.18188152>}


wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.


epoch 6 saved checkpoint: models/checkpoints/baseline/ckpt-3
epoch 7 - batch 10 - loss 1.5450986623764038 - precision 0.15190483629703522 - recall 0.11955031752586365 - f1 0.12250936031341553 - sparse_categorical_accuracy 0.27502790093421936
epoch 7 - batch 20 - loss 1.7563154697418213 - precision 0.19824402034282684 - recall 0.1375408172607422 - f1 0.1468460112810135 - sparse_categorical_accuracy 0.2767857015132904
epoch 7 - batch 30 - loss 1.6583950519561768 - precision 0.22345325350761414 - recall 0.14756028354167938 - f1 0.16051127016544342 - sparse_categorical_accuracy 0.2776413559913635
epoch 7 - batch 40 - loss 1.5171457529067993 - precision 0.24320197105407715 - recall 0.1540786623954773 - f1 0.169716477394104 - sparse_categorical_accuracy 0.2781040668487549
epoch 7 - batch 50 - loss 1.592788577079773 - precision 0.2585321366786957 - recall 0.157738596200943 - f1 0.17594601213932037 - sparse_categorical_accuracy 0.277924120426178
epoch 7 - batch 60 - loss 1.527733564376831 - pr

KeyboardInterrupt: ignored